# Import modules

In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime

------
# Filter useful dataset

## Read data sets

In [2]:
TRANSACTION = pd.read_csv("../data/raw/TRANSACTION_02_08.csv")

## Build `useful` filter index

- `useful` means that only customers who have transaction records in history would have chance to turn away. 

In [3]:
useful_filter_id = TRANSACTION[["msno"]].copy()

In [4]:
useful_filter_id = pd.DataFrame(data=pd.unique(useful_filter_id["msno"]))

In [5]:
useful_filter_id.columns = ['msno']
useful_filter_id.head()

,msno
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=
2,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=
3,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=
4,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=


## Filter MEMBERS data set

In [21]:
MEMBERS = pd.read_csv("../data/raw/MEMBERS.csv")

In [22]:
MEMBERS.head()

,msno,registration_init_time,city,bd,gender,registered_via
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,2011-09-11,1,0,NaN,11
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,2011-09-14,1,0,NaN,7
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,2011-09-15,1,0,NaN,11
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,2011-09-15,1,0,NaN,11
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,2011-09-15,6,32,female,9


In [23]:
MEMBERS_USEFUL = pd.merge(left=useful_filter_id, right=MEMBERS, on="msno", how="left")

In [24]:
del MEMBERS # save memory

### Output processed useful MEMBERS data set

In [25]:
MEMBERS_USEFUL.to_csv("../data/processed/MEMBERS_USEFUL.csv", index=False)

## Filter USER_LOGS dataset

In [33]:
UER_LOGS = pd.read_csv("../data/raw/USER_LOG_TRAIN.csv")

In [34]:
UER_LOGS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112957255 entries, 0 to 112957254
Data columns (total 9 columns):
msno          object
date          object
num_25        float64
num_50        float64
num_75        int64
num_985       int64
num_100       int64
num_unq       int64
total_secs    float64
dtypes: float64(3), int64(4), object(2)
memory usage: 7.6+ GB


### Convert `date` column to datetime object

In [35]:
UER_LOGS = UER_LOGS.dropna()

In [36]:
UER_LOGS["date"] = UER_LOGS["date"].astype("str")

In [37]:
UER_LOGS['date'] = UER_LOGS['date'].map(lambda x: datetime(int(x[:4]), int(x[5:7]), int(x[8:])))

In [38]:
UER_LOGS["date"].tail()

112957250   2017-02-23
112957251   2017-02-26
112957252   2017-02-26
112957253   2017-02-27
112957254   2017-02-28
Name: date, dtype: datetime64[ns]

###  Filter last 3 month user logs data

In [39]:
USER_LOGS_4M = UER_LOGS[UER_LOGS["date"] > datetime(year=2016, month=10, day=31)]

In [40]:
USER_LOGS_3M.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51518675 entries, 2 to 112957254
Data columns (total 9 columns):
msno          object
date          datetime64[ns]
num_25        float64
num_50        float64
num_75        int64
num_985       int64
num_100       int64
num_unq       int64
total_secs    float64
dtypes: datetime64[ns](1), float64(3), int64(4), object(1)
memory usage: 3.8+ GB


In [41]:
del UER_LOGS # save memory

In [42]:
UER_LOGS_USEFUL = pd.merge(left=useful_filter_id, right=USER_LOGS_4M, on="msno", how="left")

In [43]:
del USER_LOGS_4M

### Output processed useful USER_LOGS data set

In [44]:
UER_LOGS_USEFUL.to_csv("../data/processed/USER_LOGS_USEFUL.csv", index=False)

------
## Filter predicting data set

### Build predicting filter index

In [6]:
TRANSACTION["Exp_Year"] = TRANSACTION["membership_expire_date"].map(lambda x: int(x[:4]))
TRANSACTION["Exp_Month"] = TRANSACTION["membership_expire_date"].map(lambda x: int(x[5:7]))

In [7]:
pred_index = ((TRANSACTION["Exp_Year"] == 2017)&
              (TRANSACTION["Exp_Month"] == 2)
             )

In [8]:
pred_filter_id = TRANSACTION[["msno"]][pred_index].copy()

In [9]:
pred_filter_id = pd.DataFrame(data=pd.unique(pred_filter_id["msno"]))

In [10]:
pred_filter_id.columns = ['msno']
pred_filter_id.head()

,msno
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=


### Filter transactions of membership which expired in Feb

In [11]:
TRANSACTION_PRED = pd.merge(left=pred_filter_id, right=TRANSACTION, on="msno", how="left")

In [12]:
TRANSACTION_PRED.to_csv("../data/processed/TRANSACTION_PRED.csv", index=False)

In [13]:
del TRANSACTION_PRED

### Filter customers whose membership expired in Feb and not pay for next period yet

In [26]:
MEMBERS_PRED = pd.merge(left=pred_filter_id, right=MEMBERS_USEFUL, on="msno", how="left")

In [27]:
MEMBERS_PRED.head()

,msno,registration_init_time,city,bd,gender,registered_via
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2014-06-08,1.0,0.0,NaN,7.0
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,2010-11-18,15.0,26.0,male,9.0
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,2014-09-27,1.0,0.0,NaN,7.0
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,2016-03-15,18.0,21.0,male,7.0
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,2012-01-22,22.0,32.0,male,9.0


In [28]:
MEMBERS_PRED.to_csv("../data/processed/MEMBERS_PRED.csv", index=False)

In [29]:
del MEMBERS_PRED

### Filter those customers' listing music history

In [45]:
UER_LOGS_PRED = UER_LOGS_USEFUL[UER_LOGS_USEFUL["date"] > datetime(year=2016, month=11, day=30)]

In [46]:
UER_LOGS_PRED.to_csv("../data/processed/USER_LOGS_PRED.csv", index=False)

In [47]:
del UER_LOGS_PRED

------
## Filter training data set

In [14]:
STATUS = pd.read_csv("../data/raw/STATUS_02.csv")

In [15]:
train_filter_id = STATUS[["msno"]].copy()

In [16]:
train_filter_id = pd.DataFrame(data=pd.unique(train_filter_id["msno"]))

In [17]:
train_filter_id.columns = ['msno']
train_filter_id.head()

,msno
0,++9l7+WGOZ96gNp0TDXxHAydOlXYE0CiuqWFTxA6zZI=
1,++AH7m/EQ4iKe6wSlfO/xXAJx50p+fCeTyF90GoE9Pg=
2,++FNH/2Fg+6/Q+61X5t3PTVvNsFC699bnEVaVN6HT+o=
3,++L3Kr4/CtPSdU9BrsaX/E9ziEKoD9yK5vwk8DVXmHo=
4,++UEvwqAY2F9VpiLLHeicRU4D6FzbSDiaDrFkPrhqq8=


In [18]:
TRANSACTION_TRAIN = pd.merge(left=train_filter_id, right=TRANSACTION, on="msno", how="left")

In [ ]:
TRANSACTION_TRAIN = TRANSACTION_TRAIN[TRANSACTION_TRAIN["transaction_date"] < datetime(2017, 2, 1)]

In [19]:
TRANSACTION_TRAIN.to_csv("../data/processed/TRANSACTION_TRAIN.csv", index=False)

In [20]:
del TRANSACTION
del TRANSACTION_TRAIN

In [30]:
MEMBERS_TRAIN = pd.merge(left=train_filter_id, right=MEMBERS_USEFUL, on="msno", how="left")

In [31]:
MEMBERS_TRAIN.to_csv("../data/processed/MEMBERS_TRAIN.csv", index=False)

In [32]:
del MEMBERS_USEFUL
del MEMBERS_TRAIN

In [48]:
UER_LOGS_TRAIN = UER_LOGS_USEFUL[UER_LOGS_USEFUL["date"] < datetime(year=2017, month=2, day=1)]

In [49]:
UER_LOGS_TRAIN.to_csv("../data/processed/USER_LOGS_TRAIN.csv", index=False)

In [50]:
del UER_LOGS_USEFUL
del UER_LOGS_TRAIN